In [1]:
class Board:
    """
    Board class for Peg Solitaire.
    Manages the internal state of the board (positions of pegs and holes).
    Does NOT enforce move legality – only state management.
    """

    # קואורדינטות חוקיות בלוח 7x7, בצורה של צלב (33 חורים)
    LEGAL_POSITIONS = [
        (r, c) for r in range(7) for c in range(7)
        if not ((r < 2 or r > 4) and (c < 2 or c > 4))
    ]

    def __init__(self):
        """
        Initializes a standard Peg Solitaire board (cross pattern).
        All pegs are set except the center position.
        """
        self.state = {}
        for pos in self.LEGAL_POSITIONS:
            self.state[pos] = 1  # 1=peg, 0=hole

        self.state[(3, 3)] = 0  # Center is empty

    def get(self, pos):
        """
        Returns 1 if peg is present at position, 0 if hole, None if illegal position.
        """
        return self.state.get(pos, None)

    def set(self, pos, value):
        """
        Sets a position to 1 (peg) or 0 (hole).
        """
        if pos in self.LEGAL_POSITIONS and value in (0, 1):
            self.state[pos] = value
        else:
            raise ValueError(f"Illegal position or value: {pos}, {value}")

    def all_pegs(self):
        """
        Returns a list of all positions with pegs.
        """
        return [pos for pos, val in self.state.items() if val == 1]

    def all_holes(self):
        """
        Returns a list of all positions with holes.
        """
        return [pos for pos, val in self.state.items() if val == 0]

    def copy(self):
        """
        Returns a deep copy of the board.
        """
        new_board = Board()
        new_board.state = self.state.copy()
        return new_board

    def as_array(self):
        """
        Returns a 7x7 array with: 1=peg, 0=hole, -1=illegal position.
        Useful for neural net input.
        """
        arr = [[-1 for _ in range(7)] for _ in range(7)]
        for pos in self.LEGAL_POSITIONS:
            arr[pos[0]][pos[1]] = self.state[pos]
        return arr

    def count_pegs(self):
        """
        Returns the number of pegs currently on the board.
        """
        return sum(self.state[pos] for pos in self.LEGAL_POSITIONS)
    def to_dict(self):
        return self.state.copy()
    def __hash__(self):
        return hash(tuple(sorted(self.state.items())))

    def __eq__(self, other):
        return isinstance(other, Board) and self.state == other.state
    def set_state(self, state_dict):
        """Set board to a given dictionary of positions (for randomization/testing)."""
        for pos in self.LEGAL_POSITIONS:
            self.state[pos] = state_dict.get(pos, 0)

In [2]:
class Game:
    """
    Game logic for Peg Solitaire.
    Holds a Board, manages move legality, applying moves, undo/redo, reward, and end-of-game conditions.
    """

    DIRECTIONS = [(-2, 0), (2, 0), (0, -2), (0, 2)]

    def __init__(self, board=None, reward_fn=None):
        """
        Initializes the game.
        board: Board object to start from (default: standard).
        reward_fn: Callable (state, move, done) -> float, for RL training.
        """
        self.board = board.copy() if board else Board()
        self.move_history = []         # [(from_pos, to_pos, over_pos, Board-state-before)]
        self.redo_stack = []           # for redo functionality
        self.last_move = None
        self.reward_fn = reward_fn     # RL: custom reward function
        self.custom_metadata = {}      # extra info (for RL/debug/stats)
        self.move_log = []             # for logging all moves (can be exported)

    def is_legal_move(self, from_pos, to_pos):
        if (from_pos not in Board.LEGAL_POSITIONS) or (to_pos not in Board.LEGAL_POSITIONS):
            return (False, None)
        dr = to_pos[0] - from_pos[0]
        dc = to_pos[1] - from_pos[1]
        if (abs(dr), abs(dc)) not in [(2, 0), (0, 2)]:
            return (False, None)
        over_pos = ((from_pos[0] + to_pos[0]) // 2, (from_pos[1] + to_pos[1]) // 2)
        if (self.board.get(from_pos) != 1 or
            self.board.get(over_pos) != 1 or
            self.board.get(to_pos) != 0):
            return (False, None)
        return (True, over_pos)

    def get_legal_moves(self):
        moves = []
        for from_pos in Board.LEGAL_POSITIONS:
            if self.board.get(from_pos) != 1:
                continue
            for d in self.DIRECTIONS:
                to_pos = (from_pos[0] + d[0], from_pos[1] + d[1])
                legal, over_pos = self.is_legal_move(from_pos, to_pos)
                if legal:
                    moves.append((from_pos, to_pos, over_pos))
        return moves

    def apply_move(self, from_pos, to_pos):
        """
        Applies a legal move (if valid), records state for undo/redo.
        Returns (applied:bool, reward:float, done:bool, info:dict).
        """
        legal, over_pos = self.is_legal_move(from_pos, to_pos)
        if not legal:
            return False, 0.0, self.is_game_over(), {"reason": "Illegal move"}
        # Store history for undo: deep copy of board state
        board_before = self.board.copy()
        self.move_history.append((from_pos, to_pos, over_pos, board_before))
        self.redo_stack.clear()  # clear redo stack after new move
        # Apply move
        self.board.set(from_pos, 0)
        self.board.set(over_pos, 0)
        self.board.set(to_pos, 1)
        self.last_move = (from_pos, over_pos, to_pos)
        self.move_log.append(self.last_move)
        # Compute reward
        reward = 0.0
        done = self.is_game_over()
        if self.reward_fn:
            reward = self.reward_fn(self.board.copy(), self.last_move, done)
        info = {"last_move": self.last_move, "done": done}
        return True, reward, done, info

    def undo(self):
        """
        Undo the last move, if possible. Returns True if undone.
        """
        if not self.move_history:
            return False
        (from_pos, to_pos, over_pos, board_before) = self.move_history.pop()
        self.redo_stack.append((from_pos, to_pos, over_pos, self.board.copy()))
        self.board = board_before
        self.last_move = self.move_history[-1][:3] if self.move_history else None
        return True

    def redo(self):
        """
        Redo a move that was undone, if possible. Returns True if redone.
        """
        if not self.redo_stack:
            return False
        (from_pos, to_pos, over_pos, board_before) = self.redo_stack.pop()
        self.move_history.append((from_pos, to_pos, over_pos, board_before))
        self.board = board_before.copy()
        self.apply_move(from_pos, to_pos)
        return True

    def is_game_over(self):
        return len(self.get_legal_moves()) == 0

    def is_win(self):
        return (self.board.count_pegs() == 1) and (self.board.get((3, 3)) == 1)

    def reset(self, board=None):
        self.board = board.copy() if board else Board()
        self.move_history.clear()
        self.redo_stack.clear()
        self.last_move = None
        self.move_log.clear()
        self.custom_metadata.clear()

    def get_state(self):
        return self.board.copy()

    def set_state(self, state):
        """
        Sets board to a specific Board object or dictionary state.
        """
        if isinstance(state, Board):
            self.board = state.copy()
        elif isinstance(state, dict):
            self.board.set_state(state)
        else:
            raise ValueError("Invalid state type for set_state")
        self.move_history.clear()
        self.redo_stack.clear()
        self.last_move = None

    def __hash__(self):
        return hash(self.board)

    def __eq__(self, other):
        return isinstance(other, Game) and self.board == other.board

    def export_move_log(self):
        """
        Returns the move log (list of moves) for analysis or training.
        """
        return list(self.move_log)

    def get_custom_metadata(self, key, default=None):
        return self.custom_metadata.get(key, default)

    def set_custom_metadata(self, key, value):
        self.custom_metadata[key] = value

In [3]:
import tkinter as tk
from tkinter import messagebox

class PegSolitaireGUI(tk.Frame):
    CELL_SIZE = 60
    PEG_RADIUS = 22
    PADDING = 16

    PEG_COLOR = "#FFD600"
    HOLE_COLOR = "#202020"
    OUTLINE_COLOR = "#333"
    HIGHLIGHT_COLOR = "#42A5F5"
    BG_COLOR = "#eeeeee"

    def __init__(self, master, game: "Game"):
        super().__init__(master, bg=self.BG_COLOR)
        self.master = master
        self.game = game
        self.selected_pos = None
        self.canvas = tk.Canvas(self, width=7*self.CELL_SIZE+self.PADDING*2,
                                     height=7*self.CELL_SIZE+self.PADDING*2,
                                     bg=self.BG_COLOR, highlightthickness=0)
        self.canvas.pack()
        self.status_label = tk.Label(self, text="", font=("Arial", 16), bg=self.BG_COLOR)
        self.status_label.pack(pady=4)

        # כפתורי שליטה
        self.buttons_frame = tk.Frame(self, bg=self.BG_COLOR)
        self.buttons_frame.pack()
        self.undo_btn = tk.Button(self.buttons_frame, text="↩️ ביטול מהלך", command=self.on_undo)
        self.undo_btn.grid(row=0, column=0, padx=3)
        self.redo_btn = tk.Button(self.buttons_frame, text="↪️ קדימה", command=self.on_redo)
        self.redo_btn.grid(row=0, column=1, padx=3)
        self.reset_btn = tk.Button(self.buttons_frame, text="משחק חדש", command=self.on_reset)
        self.reset_btn.grid(row=0, column=2, padx=3)

        # לוג מהלכים
        self.move_log_label = tk.Label(self, text="לוג מהלכים:", font=("Arial", 12), bg=self.BG_COLOR)
        self.move_log_label.pack()
        self.move_log_listbox = tk.Listbox(self, height=6, width=40, font=("Consolas", 11))
        self.move_log_listbox.pack(pady=(0, 8))

        self.draw_board()
        self.canvas.bind("<Button-1>", self.on_canvas_click)
        self.update_status()
        self.update_buttons()
        self.update_move_log()

    def board_to_canvas_xy(self, pos):
        i, j = pos
        x = self.PADDING + j * self.CELL_SIZE + self.CELL_SIZE // 2
        y = self.PADDING + i * self.CELL_SIZE + self.CELL_SIZE // 2
        return x, y

    def draw_board(self):
        self.canvas.delete("all")
        for pos in Board.LEGAL_POSITIONS:
            x, y = self.board_to_canvas_xy(pos)
            val = self.game.board.get(pos)
            fill = self.PEG_COLOR if val == 1 else self.HOLE_COLOR
            outline = self.OUTLINE_COLOR
            width = 3 if self.selected_pos == pos else 1
            if self.selected_pos == pos:
                outline = self.HIGHLIGHT_COLOR
            self.canvas.create_oval(
                x - self.PEG_RADIUS, y - self.PEG_RADIUS,
                x + self.PEG_RADIUS, y + self.PEG_RADIUS,
                fill=fill, outline=outline, width=width
            )
        # הדגשה על יעדים חוקיים לפין שנבחר
        if self.selected_pos:
            moves = self.game.get_legal_moves()
            targets = [to_pos for (from_pos, to_pos, _) in moves if from_pos == self.selected_pos]
            for to_pos in targets:
                x, y = self.board_to_canvas_xy(to_pos)
                self.canvas.create_oval(
                    x - self.PEG_RADIUS // 2, y - self.PEG_RADIUS // 2,
                    x + self.PEG_RADIUS // 2, y + self.PEG_RADIUS // 2,
                    outline=self.HIGHLIGHT_COLOR, width=3
                )

    def on_canvas_click(self, event):
        col = (event.x - self.PADDING) // self.CELL_SIZE
        row = (event.y - self.PADDING) // self.CELL_SIZE
        pos = (row, col)
        if pos not in Board.LEGAL_POSITIONS:
            return
        val = self.game.board.get(pos)
        if self.selected_pos is None:
            if val == 1:
                self.selected_pos = pos
                self.update_status("בחר לאן לקפוץ")
        else:
            if pos == self.selected_pos:
                self.selected_pos = None
                self.update_status("")
                self.draw_board()
                return
            applied, reward, done, info = self.game.apply_move(self.selected_pos, pos)
            if applied:
                self.selected_pos = None
                self.update_status("מהלך חוקי! 🎉")
                self.draw_board()
                self.update_move_log()
                if self.game.is_win():
                    self.update_status("ניצחון! נשאר פיון יחיד במרכז 👑")
                elif done:
                    self.update_status("אין מהלכים חוקיים, נסה מחדש.")
            else:
                self.update_status("מהלך לא חוקי!")
            self.draw_board()
            self.update_buttons()

    def on_undo(self):
        if self.game.undo():
            self.selected_pos = None
            self.update_status("בוצע Undo")
            self.draw_board()
            self.update_move_log()
            self.update_buttons()
        else:
            self.update_status("אין מהלך לבטל.")

    def on_redo(self):
        if self.game.redo():
            self.selected_pos = None
            self.update_status("בוצע Redo")
            self.draw_board()
            self.update_move_log()
            self.update_buttons()
        else:
            self.update_status("אין מהלך לשחזר.")

    def update_status(self, msg=None):
        if msg is not None:
            self.status_label.config(text=msg)
        else:
            pegs = self.game.board.count_pegs()
            move_count = len(self.game.move_log)
            if self.game.is_win():
                self.status_label.config(text=f"ניצחון! פיון יחיד במרכז 👑")
            elif self.game.is_game_over():
                self.status_label.config(text=f"סיום – אין מהלכים חוקיים. מהלכים: {move_count}")
            else:
                self.status_label.config(text=f"פינים על הלוח: {pegs} | מהלך: {move_count}")

    def update_buttons(self):
        self.undo_btn.config(state="normal" if self.game.move_history else "disabled")
        self.redo_btn.config(state="normal" if self.game.redo_stack else "disabled")

    def update_move_log(self):
        self.move_log_listbox.delete(0, tk.END)
        log = self.game.export_move_log()
        for idx, move in enumerate(log, 1):
            from_pos, over_pos, to_pos = move
            self.move_log_listbox.insert(tk.END,
                f"{idx:2}: {from_pos} ↠ {to_pos} (דרך {over_pos})"
            )

    def on_reset(self):
        self.game.reset()
        self.selected_pos = None
        self.draw_board()
        self.update_status()
        self.update_buttons()
        self.update_move_log()

In [4]:
if __name__ == "__main__":
    from tkinter import Tk
    root = Tk()
    root.title("מחשבת – Peg Solitaire")
    game = Game()
    gui = PegSolitaireGUI(root, game)
    gui.pack()
    root.mainloop()


2025-06-21 19:24:33.830 Python[18813:1268461] +[IMKClient subclass]: chose IMKClient_Modern
2025-06-21 19:24:33.830 Python[18813:1268461] +[IMKInputSession subclass]: chose IMKInputSession_Modern
